<a href="https://colab.research.google.com/github/mabdelmaksoud53/Colab_NAMD_Suite/blob/main/MD_NAMD_suit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Hello there!**

This is a Jupyter notebook for running Molecular Dynamics (MD) simulations using NAMD (Nanoscale Molecular Dynamics) for **Protein and Ligand** systems on Google Colab. I am encourage you to watch the tutorial bellow before using this pipeline.
The main goal of this notebook is to demonstrate how to harness the power of cloud-computing to run MD simulations in a cheap and yet feasible fashion.

---
*Notes*

- **This notebook is NOT a standard protocol for MD simulations!** It is just simple MD pipeline illustrating each step of a simulation protocol.
- This notebook used input files generated by [**charmm-gui**](https://www.charmm-gui.org/) website.
- Make sure to use a GPU-enabled instance in Google Colab. Go to Runtime and Change the runtime type and select GPU as the hardware accelerator

---
**Bugs**
- If you encounter any bugs, please report the issue to https://github.com/mabdelmaksoud53/VinaDock-Notebooks/issues

**Author**
- This notebook is created by [**Mostafa S. Abd El-Maksoud**](https://github.com/mabdelmaksoud53)

---

**Introduction**

- Molecular dynamics (MD) simulations are a pivotal computational tool in the field of molecular biology, chemistry, and material science. They allow researchers to model and study the behavior of atoms and molecules over time, providing insights into the structural dynamics and interactions at an atomic level.

- MD simulations help in visualizing the movement and interaction of molecules, which is crucial for understanding biological processes such as protein folding, enzyme catalysis, and drug-receptor interactions.
Predicting Physical Properties:

- These simulations can predict physical properties of materials and biological systems, including mechanical properties, diffusion coefficients, and thermal conductivity, which are essential for material science and pharmaceutical development.
Designing New Molecules:

- In drug discovery, MD simulations assist in the rational design of new molecules by predicting their binding affinity and stability, thus accelerating the development of new therapeutics.
Elucidating Mechanisms:

- MD provides a detailed atomic-level mechanism of various phenomena such as ligand binding, conformational changes, and reaction pathways, offering insights that are often challenging to obtain through experimental methods alone.


Using Google Colab to Run MD Simulations with NAMD

- Google Colab is an excellent platform for running computationally intensive tasks such as MD simulations, especially for users who may not have access to high-performance computing resources. Colab provides free access to GPU resources, making it an attractive option for running simulations with NAMD (Nanoscale Molecular Dynamics), a high-performance MD simulation software.




# **Using Google Drive to store MD simulation data**

We can use Google Drive to read, write, and store our simulations files because Google Colab does not allow users to keep data on their computing nodes.
Therfore we will:
1.   Import Google Drive.
2.   Create a folder in your own Google Drive to store and keep data.

In [ ]:
#@title 1. Import Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted successfully")

In [ ]:
#@title 2. Creating new directory
import os
os.chdir(f'/content/drive/MyDrive')
new_dir = input("Enter the name of the new directory: ")
# Check if the directory already exists before attempting to create it.
if not os.path.exists(new_dir):
    os.mkdir(new_dir)
    print(f" '{new_dir}' created successfully.")
else:
    print(f"Directory '{new_dir}' already exists.")

os.chdir(f'/content/drive/MyDrive/{new_dir}')
!pwd

---
# **Setting the environment for MD simulation**

Firstly, we need to install softwares required for md simulation. The main softwares we will be installing are:

1.    NAMD (https://www.ks.uiuc.edu/Research/namd/) *required*
2.    VMD  (https://www.ks.uiuc.edu/Research/vmd/) *optional*
3.    CatDCD (https://www.ks.uiuc.edu/Development/MDTools/catdcd/) *optional*


In [ ]:
#@title 1. Download and unzip NAMD.
!wget https://www.ks.uiuc.edu/Research/namd/3.0b7/download/231945/NAMD_3.0b7_Linux-x86_64-multicore-CUDA.tar.gz
print("NAMD downloaded successfully.")
!tar -zxf NAMD_3.0b7_Linux-x86_64-multicore-CUDA.tar.gz
print("NAMD Unzipped successfully.")

In [ ]:
#@title 2. Download and unzip VMD.
!wget https://www.ks.uiuc.edu/Research/vmd/alpha/vmd-1.9.4a55.bin.LINUXAMD64-CUDA102-OptiX650-OSPRay185-RTXRTRT.opengl.tar.gz
!tar -zxf vmd-1.9.4a55.bin.LINUXAMD64-CUDA102-OptiX650-OSPRay185-RTXRTRT.opengl.tar.gz

In [ ]:
#@title 3. Download  and unzip CatDCD to combine the dcd files into1.log
!wget https://www.ks.uiuc.edu/Development/MDTools/catdcd/files/catdcd-4.0b.tar.gz
!tar -zxf catdcd-4.0b.tar.gz

In [ ]:
#@title Make the NAMD file executable
#@markdown Navigate the NAMD folder and copy the path of namd3
path = input("Enter the path of namd3: ")
!chmod +x {path}
print("NAMD executed successfully.")

In [ ]:
#@title Upload the MD simulation input file
#@markdown 1. upload charmm-gui zip file.
#@markdown 2. Unzip charmm-gui file.
from google.colab import files
uploaded = files.upload()
print('input files uploaded sucessfuly')
!tar -zxf charmm-gui.tgz
print('input files are ready')

In [ ]:
#@title Specify the path of NAMD input files
#@markdown Navigate to the charmm-gui folder and copy the path of NAMD folder
path = input("Enter the path of namd: ")
os.chdir(f'{path}')

---
# **Purpose and Importance of the Equilibration Step**
- The Equilibration Step in MD Simulations is a critical preparatory step that ensures the system reaches a stable state before production runs. The main goal of this step is to relax any initial structural or energetic inconsistencies and to allow the system to adapt to the simulation conditions, such as temperature and pressure, set by the researcher.
- Throughout the equilibration process, key parameters such as temperature, pressure, and potential energy are monitored to ensure they stabilize and reach desired values. Validation of these parameters confirms that the system is well-equilibrated and ready for the production run.

In [ ]:
#@title Generate equlibration script
#@markdown Create automated Sh script to run the equilibration step
sh_file_name = input('Enter the name of the .sh file (without extension): ') + '.sh'
# Prompt the user for the ligand directory path
path_to_NAMD_executable_file = input("Enter the path of namd3: ")
path_to_inputs = input("Enter the path of input file: ")
path_to_output = input("Enter the path of output file: ")

# Define the content of the .sh file
sh_content = f"""#!/bin/bash
{path_to_NAMD_executable_file} +auto-provision {path_to_inputs} > {path_to_output}
 """

# # Write the content to the .sh file
with open(sh_file_name, 'w') as file:
     file.write(sh_content)

# Make the .sh file executable
import os
os.chmod(sh_file_name, 0o755)

print(f"{sh_file_name} created and made executable.")


In [ ]:
#@title Run the equilibration script
#@markdown enter the name of the file you created in the previous step
script_name = input("Enter the name of the script ")
!bash $script_name
print("Equilibration step completed successfully.")

---
# **Purpose and importance of the Production Step**
- After the successful completion of the equilibration step, MD simulation proceeds to the production step. This phase is critical as it generates the actual data used for analyzing and understanding the molecular system under study.
- By carefully executing the production step, researchers can obtain comprehensive and accurate data that provide deep insights into the molecular dynamics and properties of the system under study.
- This step is crucial for translating the detailed, atomistic simulations into meaningful scientific knowledge and practical applications.

In [ ]:
#@title Generate production script
#@markdown Create automated Sh script to run the equilibration step
sh_file_name = input('Enter the name of the .sh file (without extension): ') + '.sh'
# Prompt the user for the ligand directory path
path_to_NAMD_executable_file = input("Enter the path of namd3: ")
path_to_inputs = input("Enter the path of input file: ")
path_to_output = input("Enter the path of output file: ")

# Define the content of the .sh file
sh_content = f"""#!/bin/bash
{path_to_NAMD_executable_file} +auto-provision {path_to_inputs} > {path_to_output}
 """

# # Write the content to the .sh file
with open(sh_file_name, 'w') as file:
     file.write(sh_content)

# Make the .sh file executable
import os
os.chmod(sh_file_name, 0o755)

print(f"{sh_file_name} created and made executable.")


In [ ]:
#@title Run the production script
#@markdown enter the name of the file you created in the previous step
script_name = input("Enter the name of the script ")
!bash $script_name
print("Production step completed successfully.")